In [1]:
import json 
import numpy as np 
import tensorflow as tf
import nltk

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.preprocessing import LabelEncoder

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mannatsaluja/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mannatsaluja/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mannatsaluja/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [3]:
training_sentences = []
training_labels = []
labels = []
responses = []

In [4]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        tokens = word_tokenize(pattern)
        tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]
        training_sentences.append(tokens)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [5]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [6]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [7]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [8]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 80)                1360      
                                                                 
Total params: 17904 (69.94 KB)
Trainable params: 17904 (69.94 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

In [10]:
epochs = 650
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs, validation_split=0.1)

Epoch 1/650
7/7 [==============================] - 0s 12ms/step - loss: 4.3811 - accuracy: 0.0245 - val_loss: 4.3872 - val_accuracy: 0.0000e+00
Epoch 2/650
7/7 [==============================] - 0s 2ms/step - loss: 4.3751 - accuracy: 0.0343 - val_loss: 4.3932 - val_accuracy: 0.0000e+00
Epoch 3/650
7/7 [==============================] - 0s 2ms/step - loss: 4.3698 - accuracy: 0.0294 - val_loss: 4.3996 - val_accuracy: 0.0000e+00
Epoch 4/650
7/7 [==============================] - 0s 2ms/step - loss: 4.3635 - accuracy: 0.0392 - val_loss: 4.4073 - val_accuracy: 0.0000e+00
Epoch 5/650
7/7 [==============================] - 0s 2ms/step - loss: 4.3562 - accuracy: 0.0196 - val_loss: 4.4167 - val_accuracy: 0.0000e+00
Epoch 6/650
7/7 [==============================] - 0s 2ms/step - loss: 4.3467 - accuracy: 0.0196 - val_loss: 4.4288 - val_accuracy: 0.0000e+00
Epoch 7/650
7/7 [==============================] - 0s 2ms/step - loss: 4.3347 - accuracy: 0.0294 - val_loss: 4.4452 - val_accuracy: 0.0000e+0

Epoch 58/650
7/7 [==============================] - 0s 2ms/step - loss: 3.4686 - accuracy: 0.0931 - val_loss: 7.2087 - val_accuracy: 0.0000e+00
Epoch 59/650
7/7 [==============================] - 0s 2ms/step - loss: 3.4506 - accuracy: 0.0931 - val_loss: 7.3155 - val_accuracy: 0.0000e+00
Epoch 60/650
7/7 [==============================] - 0s 2ms/step - loss: 3.4310 - accuracy: 0.0931 - val_loss: 7.4089 - val_accuracy: 0.0000e+00
Epoch 61/650
7/7 [==============================] - 0s 2ms/step - loss: 3.4152 - accuracy: 0.0931 - val_loss: 7.5296 - val_accuracy: 0.0000e+00
Epoch 62/650
7/7 [==============================] - 0s 2ms/step - loss: 3.3954 - accuracy: 0.0931 - val_loss: 7.6442 - val_accuracy: 0.0000e+00
Epoch 63/650
7/7 [==============================] - 0s 2ms/step - loss: 3.3796 - accuracy: 0.0931 - val_loss: 7.7836 - val_accuracy: 0.0000e+00
Epoch 64/650
7/7 [==============================] - 0s 2ms/step - loss: 3.3620 - accuracy: 0.0931 - val_loss: 7.9369 - val_accuracy: 0.0

7/7 [==============================] - 0s 2ms/step - loss: 2.3897 - accuracy: 0.3775 - val_loss: 15.1102 - val_accuracy: 0.0000e+00
Epoch 115/650
7/7 [==============================] - 0s 2ms/step - loss: 2.3724 - accuracy: 0.3873 - val_loss: 15.2738 - val_accuracy: 0.0000e+00
Epoch 116/650
7/7 [==============================] - 0s 2ms/step - loss: 2.3530 - accuracy: 0.3725 - val_loss: 15.4465 - val_accuracy: 0.0000e+00
Epoch 117/650
7/7 [==============================] - 0s 2ms/step - loss: 2.3336 - accuracy: 0.3873 - val_loss: 15.6693 - val_accuracy: 0.0000e+00
Epoch 118/650
7/7 [==============================] - 0s 2ms/step - loss: 2.3151 - accuracy: 0.3922 - val_loss: 15.8843 - val_accuracy: 0.0000e+00
Epoch 119/650
7/7 [==============================] - 0s 2ms/step - loss: 2.3004 - accuracy: 0.3873 - val_loss: 16.0111 - val_accuracy: 0.0000e+00
Epoch 120/650
7/7 [==============================] - 0s 2ms/step - loss: 2.2743 - accuracy: 0.3873 - val_loss: 16.2421 - val_accuracy: 0.0

7/7 [==============================] - 0s 2ms/step - loss: 1.4912 - accuracy: 0.5588 - val_loss: 25.3684 - val_accuracy: 0.0000e+00
Epoch 171/650
7/7 [==============================] - 0s 2ms/step - loss: 1.4790 - accuracy: 0.5686 - val_loss: 25.6669 - val_accuracy: 0.0000e+00
Epoch 172/650
7/7 [==============================] - 0s 2ms/step - loss: 1.4658 - accuracy: 0.5735 - val_loss: 25.8367 - val_accuracy: 0.0000e+00
Epoch 173/650
7/7 [==============================] - 0s 2ms/step - loss: 1.4613 - accuracy: 0.6029 - val_loss: 26.0284 - val_accuracy: 0.0000e+00
Epoch 174/650
7/7 [==============================] - 0s 2ms/step - loss: 1.4420 - accuracy: 0.5980 - val_loss: 26.1809 - val_accuracy: 0.0000e+00
Epoch 175/650
7/7 [==============================] - 0s 2ms/step - loss: 1.4365 - accuracy: 0.5882 - val_loss: 26.3531 - val_accuracy: 0.0000e+00
Epoch 176/650
7/7 [==============================] - 0s 2ms/step - loss: 1.4249 - accuracy: 0.6176 - val_loss: 26.5803 - val_accuracy: 0.0

7/7 [==============================] - 0s 2ms/step - loss: 0.9885 - accuracy: 0.7451 - val_loss: 35.6791 - val_accuracy: 0.0000e+00
Epoch 227/650
7/7 [==============================] - 0s 2ms/step - loss: 0.9780 - accuracy: 0.7647 - val_loss: 35.9017 - val_accuracy: 0.0000e+00
Epoch 228/650
7/7 [==============================] - 0s 2ms/step - loss: 0.9701 - accuracy: 0.7843 - val_loss: 36.1578 - val_accuracy: 0.0000e+00
Epoch 229/650
7/7 [==============================] - 0s 2ms/step - loss: 0.9631 - accuracy: 0.7794 - val_loss: 36.2377 - val_accuracy: 0.0000e+00
Epoch 230/650
7/7 [==============================] - 0s 2ms/step - loss: 0.9589 - accuracy: 0.7598 - val_loss: 36.4764 - val_accuracy: 0.0000e+00
Epoch 231/650
7/7 [==============================] - 0s 2ms/step - loss: 0.9545 - accuracy: 0.7647 - val_loss: 36.7527 - val_accuracy: 0.0000e+00
Epoch 232/650
7/7 [==============================] - 0s 2ms/step - loss: 0.9462 - accuracy: 0.7598 - val_loss: 36.8445 - val_accuracy: 0.0

...

7/7 [==============================] - 0s 2ms/step - loss: 0.2760 - accuracy: 0.9608 - val_loss: 62.9456 - val_accuracy: 0.0000e+00
Epoch 395/650
7/7 [==============================] - 0s 2ms/step - loss: 0.2751 - accuracy: 0.9559 - val_loss: 63.0651 - val_accuracy: 0.0000e+00
Epoch 396/650
7/7 [==============================] - 0s 2ms/step - loss: 0.2738 - accuracy: 0.9510 - val_loss: 63.1621 - val_accuracy: 0.0000e+00
Epoch 397/650
7/7 [==============================] - 0s 2ms/step - loss: 0.2711 - accuracy: 0.9461 - val_loss: 63.4324 - val_accuracy: 0.0000e+00
Epoch 398/650
7/7 [==============================] - 0s 2ms/step - loss: 0.2685 - accuracy: 0.9461 - val_loss: 63.5024 - val_accuracy: 0.0000e+00
Epoch 399/650
7/7 [==============================] - 0s 2ms/step - loss: 0.2679 - accuracy: 0.9510 - val_loss: 63.5217 - val_accuracy: 0.0000e+00
Epoch 400/650
7/7 [==============================] - 0s 2ms/step - loss: 0.2678 - accuracy: 0.9510 - val_loss: 63.7838 - val_accuracy: 0.0

7/7 [==============================] - 0s 2ms/step - loss: 0.1864 - accuracy: 0.9804 - val_loss: 69.6264 - val_accuracy: 0.0000e+00
Epoch 451/650
7/7 [==============================] - 0s 2ms/step - loss: 0.1838 - accuracy: 0.9804 - val_loss: 69.7802 - val_accuracy: 0.0000e+00
Epoch 452/650
7/7 [==============================] - 0s 2ms/step - loss: 0.1844 - accuracy: 0.9755 - val_loss: 69.9063 - val_accuracy: 0.0000e+00
Epoch 453/650
7/7 [==============================] - 0s 2ms/step - loss: 0.1827 - accuracy: 0.9755 - val_loss: 69.9268 - val_accuracy: 0.0000e+00
Epoch 454/650
7/7 [==============================] - 0s 2ms/step - loss: 0.1805 - accuracy: 0.9706 - val_loss: 69.9892 - val_accuracy: 0.0000e+00
Epoch 455/650
7/7 [==============================] - 0s 2ms/step - loss: 0.1800 - accuracy: 0.9706 - val_loss: 70.1908 - val_accuracy: 0.0000e+00
Epoch 456/650
7/7 [==============================] - 0s 2ms/step - loss: 0.1797 - accuracy: 0.9755 - val_loss: 70.3150 - val_accuracy: 0.0

7/7 [==============================] - 0s 2ms/step - loss: 0.1271 - accuracy: 0.9804 - val_loss: 75.3085 - val_accuracy: 0.0000e+00
Epoch 507/650
7/7 [==============================] - 0s 2ms/step - loss: 0.1220 - accuracy: 0.9853 - val_loss: 75.2191 - val_accuracy: 0.0000e+00
Epoch 508/650
7/7 [==============================] - 0s 2ms/step - loss: 0.1229 - accuracy: 0.9853 - val_loss: 75.2705 - val_accuracy: 0.0000e+00
Epoch 509/650
7/7 [==============================] - 0s 2ms/step - loss: 0.1218 - accuracy: 0.9804 - val_loss: 75.6168 - val_accuracy: 0.0000e+00
Epoch 510/650
7/7 [==============================] - 0s 2ms/step - loss: 0.1221 - accuracy: 0.9853 - val_loss: 75.4549 - val_accuracy: 0.0000e+00
Epoch 511/650
7/7 [==============================] - 0s 2ms/step - loss: 0.1204 - accuracy: 0.9853 - val_loss: 75.4587 - val_accuracy: 0.0000e+00
Epoch 512/650
7/7 [==============================] - 0s 2ms/step - loss: 0.1200 - accuracy: 0.9853 - val_loss: 75.6957 - val_accuracy: 0.0

7/7 [==============================] - 0s 2ms/step - loss: 0.0860 - accuracy: 0.9853 - val_loss: 79.8495 - val_accuracy: 0.0000e+00
Epoch 563/650
7/7 [==============================] - 0s 2ms/step - loss: 0.0854 - accuracy: 0.9902 - val_loss: 80.0333 - val_accuracy: 0.0000e+00
Epoch 564/650
7/7 [==============================] - 0s 2ms/step - loss: 0.0847 - accuracy: 0.9902 - val_loss: 80.1000 - val_accuracy: 0.0000e+00
Epoch 565/650
7/7 [==============================] - 0s 2ms/step - loss: 0.0855 - accuracy: 0.9804 - val_loss: 80.2238 - val_accuracy: 0.0000e+00
Epoch 566/650
7/7 [==============================] - 0s 2ms/step - loss: 0.0840 - accuracy: 0.9902 - val_loss: 80.2132 - val_accuracy: 0.0000e+00
Epoch 567/650
7/7 [==============================] - 0s 2ms/step - loss: 0.0845 - accuracy: 0.9853 - val_loss: 80.3699 - val_accuracy: 0.0000e+00
Epoch 568/650
7/7 [==============================] - 0s 2ms/step - loss: 0.0840 - accuracy: 0.9853 - val_loss: 80.5947 - val_accuracy: 0.0

7/7 [==============================] - 0s 2ms/step - loss: 0.0597 - accuracy: 0.9902 - val_loss: 84.1031 - val_accuracy: 0.0000e+00
Epoch 619/650
7/7 [==============================] - 0s 2ms/step - loss: 0.0603 - accuracy: 0.9902 - val_loss: 84.2443 - val_accuracy: 0.0000e+00
Epoch 620/650
7/7 [==============================] - 0s 2ms/step - loss: 0.0601 - accuracy: 0.9853 - val_loss: 84.3820 - val_accuracy: 0.0000e+00
Epoch 621/650
7/7 [==============================] - 0s 2ms/step - loss: 0.0605 - accuracy: 0.9853 - val_loss: 84.5744 - val_accuracy: 0.0000e+00
Epoch 622/650
7/7 [==============================] - 0s 2ms/step - loss: 0.0618 - accuracy: 0.9853 - val_loss: 84.3610 - val_accuracy: 0.0000e+00
Epoch 623/650
7/7 [==============================] - 0s 2ms/step - loss: 0.0586 - accuracy: 0.9902 - val_loss: 84.3701 - val_accuracy: 0.0000e+00
Epoch 624/650
7/7 [==============================] - 0s 2ms/step - loss: 0.0590 - accuracy: 0.9902 - val_loss: 84.5110 - val_accuracy: 0.0

In [11]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


INFO:tensorflow:Assets written to: chat_model/assets


In [12]:
# Accuracy of the model 
loss, accuracy = model.evaluate(padded_sequences, np.array(training_labels), verbose=0)

print("Accuracy:", accuracy)

Accuracy: 0.8942731022834778
